# Bayesian Nash Equilibrium in double bi-matrix games

This notebook introduces a solution method suggested by William Spaniel for analyzing bimatrix games where one of the players can have multiple types in this video: https://youtu.be/E0_CA9TwZ8c. It is recommended to check the video out in order to fully understand how and why the method works.

In [1]:
import numpy as np 
import nashpy
import bimatrix

Player types 
* Player 1: always a stag hunt type, 
* Player 2: with prob. `p`, she is PD, and `1-p`, she is SH. 

Payoffs: a payoff matrix *list* for each player: one payoff matrix for each type that player 2 can have. 

In [2]:
def compute_full_matrix(U1, U2, p, action_names=None): 
    '''
        Assumes that only player 2's type varies 
        (this means that player 1 has one action per row in U1, 
         while 2 has nA2**2 (one choice per type))
        Both players have one utility matrix for each realization 
        of player 2's type. 
         
        INPUTS: 
            U1: list of 2 payoff matrices for player 1 (row player)
            U2: list of 2 payoff matrices for player 2 (column player)
            p: (scalar) Probability that player 2 is the first type 
            action_names: [optional] 2-list of names of actions (nA1 and nA2 long)
        OUTPUTS: 
            t1, t2: wide-form payoff matrices suitable for finding the NE 
            A1, A2: names of actions 
    '''
    assert len(U1) == 2
    assert len(U2) == 2 
    assert np.isscalar(p)
    nA1, nA2 = U1[0].shape
    
    t1 = np.empty((nA1, nA2*nA2))
    t2 = np.empty((nA1, nA2*nA2))
    
    # player 1 chooses an action without knowing what type 2 is 
    for ia1 in range(nA1): 
        i_col = 0 
        
        # player 2 chooses an action conditional on observing her type 
        for a2_1 in range(nA2): 
            for a2_2 in range(nA2): 
                t1[ia1,i_col] = p * U1[0][ia1,a2_1] + (1.-p) * U1[1][ia1,a2_2]
                t2[ia1,i_col] = p * U2[0][ia1,a2_1] + (1.-p) * U2[1][ia1,a2_2]
                
                i_col += 1
                
    if action_names is None: 
        A1 = [f'{i}' for i in range(nA1)]
        A2 = [f'{a}{b}' for a in range(nA2) for b in range(nA2)]
    else: 
        assert len(action_names) == 2 
        A1 = action_names[0]
        assert len(A1) == nA1, f'Incorrect # of action names'
        a2 = action_names[1]
        assert len(a2) == nA2, f'Incorrect # of action names'
        
        A2 = [f'{a}{b}' for a in a2 for b in a2]
        
    return t1, t2, A1, A2

# Løsning af Question 6 PS9
### Swapping P1 and P2 to use function
This means that P1 now has a type and P2 does not observe type

In [39]:
# Pr(player 2 is the PD type)
p = 0.5

# player 1 
u1  = np.array([[1,0], [0,0]])
u2  = np.array([[0,0], [0,2]])

U1 = [u1, u2] # player 1 has same payoffs regardless of 2's type 
a1 = ['T', 'B']
A1 = [f'{a}{b}' for a in a1 for b in a1]

# player 2
u21  = np.array([[1,0], [0,0]])
u22  = np.array([[0,0], [0,2]])
U2 = [u21, u22]
A2 = ['L', 'R']


t2, t1, A2, A1 = compute_full_matrix(np.array(U2).T, np.array(U1).T, p, action_names=[A2, a1])
print(A1)
print(A2)
print(t1.T)
print(t2.T)


['TT', 'TB', 'BT', 'BB']
['L', 'R']
[[0.5 0. ]
 [0.5 1. ]
 [0.  0. ]
 [0.  1. ]]
[[0.5 0. ]
 [0.5 1. ]
 [0.  0. ]
 [0.  1. ]]


In [40]:
bimatrix.print_payoffs([t1.T, t2.T], [A1,  A2], 3)

,L,R
TT,"(0.5, 0.5)","(0.0, 0.0)"
TB,"(0.5, 0.5)","(1.0, 1.0)"
BT,"(0.0, 0.0)","(0.0, 0.0)"
BB,"(0.0, 0.0)","(1.0, 1.0)"


In [41]:
G = nashpy.Game(t1.T, t2.T)

eqs = list(G.support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i,eq in enumerate(eqs): 
    print(f'{i+1}: s1 = {eq[0]}, s2 = {eq[1]}')

Found 3 equilibria
1: s1 = [1. 0. 0. 0.], s2 = [1. 0.]
2: s1 = [0. 1. 0. 0.], s2 = [0. 1.]
3: s1 = [0. 0. 0. 1.], s2 = [0. 1.]
